<a href="https://colab.research.google.com/github/sayandas96476/RAG/blob/main/dense_retrieval_allenai_longformer_base_4096.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I have created externel knowledge base source using Wikipedia

I have used BeautifulSoup to parse the content from wikipedia based title which I reived using requests library

Then the document is preprocessed. I have used allenai/longformer-base-4096 as the embedding model as we were getting large chunk size

We have created Chroma as our vector database.Then stored it in google drive location

In the next step we load the vector database and extract top 10 most similar records



In [2]:
!pip install -q chromadb
import chromadb


In [4]:

!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00


In [5]:

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

In [7]:

embedding_function = SentenceTransformerEmbeddings(model_name= "intfloat/e5-large")

<ipython-input-7-cb5e97aa443e>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name= "intfloat/e5-large")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datase

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [8]:
embedding_function = SentenceTransformerEmbeddings(model_name= "allenai/longformer-base-4096")


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader

In [182]:
import requests
from bs4 import BeautifulSoup

def get_full_wikipedia_content(title):
    # Construct the URL for the full Wikipedia page
    url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"

    try:
        # Make the request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content div
        content_div = soup.find(id="mw-content-text")

        # Extract all paragraphs
        paragraphs = content_div.find_all('p')

        # Combine all paragraph texts
        full_text = '\n\n'.join([para.get_text() for para in paragraphs])

        # Remove citations [1], [2], etc.
        import re
        full_text = re.sub(r'\[\d+\]', '', full_text)

        return full_text.strip()

    except requests.RequestException as e:
        return f"Error fetching page: {str(e)}"

# First install beautifulsoup4 if you haven't:
# pip install beautifulsoup4

# Example usage
title = "Batman"
text = get_full_wikipedia_content(title)


In [ ]:
text = text.replace('\n', '')

lis = text.split(".")
def combine_strings(original_list, chunk_size=3):
    return [''.join(original_list[i:i + chunk_size])
            for i in range(0, len(original_list), chunk_size)]

# Example usage:
original = lis  # Your 100 strings
combined = combine_strings(original)
text = """ """
for i in combined:
  text += i+"\n\n\n"


In [193]:

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = '\n\n\n',
    chunk_size = 700,
)

texts = text_splitter.create_documents([text])  # Remove the square brackets


In [194]:
len(texts)

109

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [195]:

persist_directory = "/content/drive/MyDrive/testing/db__longform_1"
vectordb = Chroma.from_documents(documents=texts, embedding=embedding_function, persist_directory=persist_directory)


In [196]:

vectordb = Chroma(
    persist_directory="/content/drive/MyDrive/testing/db__longform_1",
    embedding_function=embedding_function
)


In [206]:
# Now you can perform queries
query = "Who created Batman comic"

# Method 1: Basic similarity search
docs = vectordb.similarity_search(query, k=15)  # k is the number of results you want


In [207]:
for doc in docs:
    print("===========")
    print(doc)

page_content='More recent developments under DC Rebirth and Infinite Frontier have continued to evolve Batman's universe, exploring new characters like Gotham and Gotham Girl, and tackling contemporary issues within the context of Gotham City's ever-evolving landscape of crime and heroismBatman's secret identity is Bruce Wayne, a wealthy American industrialist As a child, Bruce witnessed the murder of his parents, Dr'
page_content='His gloves typically feature three scallops that protrude from long, gauntlet-like cuffs, although in his earliest appearances he wore short, plain gloves without the scallops The overall look of the character, particularly the length of the cowl's ears and of the cape, varies greatly depending on the artist Dennis O'Neil said, "We now say that Batman has two hundred suits hanging in the Batcave so they don't have to look the same 


Everybody loves to draw Batman, and everybody wants to put their own spin on it'
page_content='Batman's history has undergone 

In [204]:
texts[9]

Document(metadata={}, page_content="Early stories were dark, featuring a Batman who did not shy away from killing The character quickly became popular, leading to his own solo title in 1940 Robin, Batman's sidekick, was introduced in 1940, lightening the tone and boosting sales\n\n\n Over the next few years, Batman's rogues' gallery expanded with iconic villains like the Joker and CatwomanThe 1950s saw Batman in lighter, science fiction-influenced stories However, declining sales led to a 1964 revamp by editor Julius Schwartz, who returned Batman to his detective roots and updated his appearance")

In [208]:
# Now you can perform queries
query = "Who was Batman's first love"

# Method 1: Basic similarity search
docs = vectordb.similarity_search(query, k=10)  # k is the number of results you want


In [209]:
for doc in docs:
    print(doc)

page_content='More recent developments under DC Rebirth and Infinite Frontier have continued to evolve Batman's universe, exploring new characters like Gotham and Gotham Girl, and tackling contemporary issues within the context of Gotham City's ever-evolving landscape of crime and heroismBatman's secret identity is Bruce Wayne, a wealthy American industrialist As a child, Bruce witnessed the murder of his parents, Dr'
page_content='Continuity was not important in those days"The driving force behind Bruce Wayne's character is his parents' murder and their absence Bob Kane and Bill Finger discussed Batman's background and decided that "there's nothing more traumatic than having your parents murdered before your eyes"'
page_content='His gloves typically feature three scallops that protrude from long, gauntlet-like cuffs, although in his earliest appearances he wore short, plain gloves without the scallops The overall look of the character, particularly the length of the cowl's ears and of

In [49]:
len(texts)

35